In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
X_in = np.load('inputs.npy')

In [3]:
input_size = X_in.shape[1]
n_classes = 2 # only two, as blue/not blue

In [4]:
K = 4 # number of neurons in the first hidden layer; should be between input size and number of classes

In [5]:
X = tf.placeholder(np.float64, [None, input_size])
Y_ = tf.placeholder(np.float64, [None, n_classes])
pkeep = tf.placeholder(tf.float64)

W1 = tf.Variable(tf.truncated_normal([input_size, K], stddev=0.1, dtype=np.float64))
B1 = tf.Variable(tf.ones([K], dtype=np.float64)/10)
W3 = tf.Variable(tf.truncated_normal([K, n_classes], stddev=0.1, dtype=np.float64))
B3 = tf.Variable(tf.zeros([n_classes], dtype=np.float64))

In [6]:
Y1 = tf.nn.tanh(tf.matmul(X, W1) + B1)
Y2 = tf.nn.dropout(Y1, pkeep) # dropout a number of neurons to prevent overfitting

In [7]:
Ylogits = tf.matmul(Y2, W3) + B3
Y3 = tf.nn.softmax(Ylogits)

In [8]:
correct_prediction = tf.equal(tf.argmax(Y3, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

In [9]:
prediction = tf.argmax(Y3, 1)

In [10]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

In [11]:
learning_rate = 1e-2
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [12]:
# possibly allow to train on the data already produced by the SVM? But the problem is that we don't get to see behind the model so it's not particularily usefl
init = tf.global_variables_initializer()
with tf.Session() as sess:
    seed = 1
    # TODO: shuffle order and retrain network on shuffled data
    sess.run(init)
    X_in = np.load('inputs.npy')
    Y_in = np.load('eyes.npy')
    
    X_train, X_test, y_train, y_test = train_test_split(X_in, Y_in, test_size=0.2)
    
    num_splits = 2
    n_epochs = 100
    
    for epoch in range(n_epochs):
        sess.run(train_step, {X: X_train, Y_: y_train, pkeep: .75})

        a, c = sess.run([accuracy, cross_entropy], feed_dict={X: X_train, Y_: y_train, pkeep: 1})
        print "\nEpoch:", epoch + 1, "Train accuracy:", a, "Loss:", c
        
        a, c, p = sess.run([accuracy, cross_entropy, prediction], feed_dict={X: X_test, Y_: y_test, pkeep: 1})
        print "Epoch:", epoch + 1, "Test accuracy:", a, "Loss:", c
        predictions = zip([np.argmax(pred) for pred in p], [np.argmax(item) for item in y_test])
        print "Test results:", "equal:", [prediction[0] == prediction[1] for prediction in predictions]
        
    print "Final:"
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: X_test, Y_: y_test, pkeep: 1})
    print "Accuracy on all data: ", a, "loss:", c



Epoch: 1 Train accuracy: 0.564516129032 Loss: 68.8288988958
Epoch: 1 Test accuracy: 0.625 Loss: 66.193599809
Test results: equal: [False, False, False, False, True, True, True, False, True, False, True, False, False, False, True, False]

Epoch: 2 Train accuracy: 0.564516129032 Loss: 68.6394801198


TypeError: Fetch argument 0 has invalid type <type 'numpy.int64'>, must be a string or Tensor. (Can not convert a int64 into a Tensor or Operation.)